In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

from config import *

In [ ]:
n_cont = 2
n_cat = [5, 6]
cat_names = ['围岩等级', '岩性']
context = Context('amended-3-7-extra', 
                  fn_cycles='tbmData/extra_data',
                  debug=0, dep_var=dep_var2 + cat_names, mulr=7, gpu_start=6, load_data=False)

In [ ]:
fmtr = DataFormatter(context, cycle_feathers=context.fn_cycles)
fmtr.cycles = fmtr.cycles.iloc[:,:-1]
# fmtr.drop_redundent_columns() # already done for cycles
idx = fmtr.beginning_index()

dropped = np.load('tbmData/cycles-removed-redundent-columns/dropped.npy').tolist()
fmtr.cycles.drop(columns=dropped+['EP2次数设置', '刀盘喷水增压泵压力'], inplace=True, errors='ignore')

In [ ]:
extra_x_var = ['桩号'] + dep_var1
df_conts, stat_x = fmtr.get_x(normalize=True)#, stat_x=stat_x)

In [ ]:
stat_x_old = np.load('tbmData/feather/amended-3-7.npz')['stat_x']
for i in range(2):
    stat_x[i][:] = stat_x_old[i]

In [ ]:
extra_x_var = ['桩号'] + dep_var1
df_conts, stat_x = fmtr.get_x(normalize=True, stat_x=stat_x)

In [ ]:
# extra_x_var = ['桩号'] + dep_var1
# df_conts, stat_x = fmtr.get_x(normalize=True)
extra_x = fmtr.get_extra_x_task2()
deps = fmtr.get_y()
stat_y = [o.loc[context.dep_var] for o in stat_x]
stat_extra_x = [o.loc[extra_x_var] for o in stat_x]
train_cont = flatten_and_cat(df_conts, [extra_x, deps], sl=context.sl)
train_cont = train_cont.iloc[:,:-2]

In [ ]:
# load original
context = Context('amended-3-7', debug=0, dep_var=dep_var2, mulr=7, gpu_start=9)

In [ ]:
train_cont_full = pd.concat([train_cont, context.cyc_cont], 0, ignore_index=True, sort=False)

In [ ]:
context.fn_feather

In [ ]:
context.fn_np

In [ ]:
train_cont_full.to_feather('tbmData/feather/amended-3-7-extra.feather')
np.savez_compressed('tbmData/feather/amended-3-7-extra.npz', idx=idx, stat_x=stat_x, stat_y=stat_y, stat_extra_x=stat_extra_x)
context.fn_feather

# generate -cat

In [ ]:
# super hacky
fn_cycles = ('no',
             list(Path('tbmData/cycles-removed-redundent-columns').glob('cycle*')) + list(Path('tbmData/extra_data/').glob('cycle*')))

In [ ]:
# context = Context('amended-1-no_noise', debug=0, dep_var=dep_var2, mulr=1, gpu_start=9)
context = Context('amended-3-7-extra', debug=0, dep_var=dep_var2, mulr=7, gpu_start=6, fn_cycles=fn_cycles)
databunch = MultiDeptTabularDataBunch.from_df('tmp', context.cyc_cont, context.dep_var, valid_idx=context.valid_idx_tile, bs=context.bs)

In [ ]:
# read context and append 
geo_raw = pd.read_csv('/home/wenxiang/datasets/tbm/geo_csv/geo2.1.csv', index_col=0)
geo_cols = ['起始桩号', '最后桩号', '实际开挖围岩类别', '洞室部位主要岩性']
geo = geo_raw.loc[:,geo_cols]
# clean nan
geo = geo.loc[geo.iloc[:,-1].notna()]
zh = context.cyc_cont.iloc[:,-5]
cat_names = geo.columns.tolist()

idx_dep_var3 = []
missing_idx = -1 #geo.shape[0] # last item + 1
for z in list(zh):
    is_within = ((z - geo.iloc[:,0]) * (z - geo.iloc[:,1])) < 0
    if is_within.any():
        idx_dep_var3.append(np.argmax(is_within.values))
    else:
        idx_dep_var3.append(missing_idx)

dep_var3 = []
for i in idx_dep_var3:
    dep_var3.append(geo.iloc[i,2:].tolist() if i != missing_idx else ['oor', 'oor'])

cycles = pd.concat([context.cyc_cont, pd.DataFrame(dep_var3, columns=['围岩等级', '岩性'])], axis=1)
cat_names = cycles.columns[-2:]
cont_names = cycles.columns[:-2]

# simplify
cat_1 = cycles.loc[:,cat_names[0]]
III_idx = np.logical_or(cat_1 == 'Ⅲa', cat_1 == 'Ⅲb')
cycles.loc[III_idx,cat_names[0]] = 'Ⅲ'
IV_idx = np.logical_or(cat_1 == 'Ⅳ好', cat_1 == 'Ⅳ差')
cycles.loc[IV_idx,cat_names[0]] = 'Ⅳ'

cat_2 = cycles.loc[:,cat_names[1]]
idx = cat_2 == '花岗岩，不连续分布有三叠系凝灰岩，累计55m'
cycles.loc[idx,cat_names[1]] = '花岗岩'

cf = Categorify(cat_names, cont_names)
cf.apply_train(cycles)

fn_feather = Path(context.fn_feather)
new_fn_feather = fn_feather.parent / (fn_feather.stem + '-cat' + fn_feather.suffix)
cycles.to_feather(new_fn_feather)
# copy npz manually

In [ ]:
decycles.iloc[:,-4]

# run

In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

from config import *

In [ ]:
n_cont = 2
n_cat = [5, 6]
cat_names = ['围岩等级', '岩性']

# context = Context('amended-1-no_noise', debug=0, dep_var=dep_var2, mulr=1, gpu_start=9)
context = Context('amended-3-7-extra-cat', debug=0, dep_var=dep_var2 + cat_names, mulr=7, gpu_start=5)
cycles = context.cyc_cont

# un-categorify
cat_columns = []
for cat_name in cat_names:
    cat_columns.append(cycles.loc[:,cat_name])
    cycles.loc[:,cat_name] = cycles.loc[:,cat_name].cat.codes.values
    
databunch = MultiDeptTabularDataBunch.from_df('tmp', cycles, context.dep_var, valid_idx=context.valid_idx_tile, bs=context.bs)

# model

In [ ]:
hyper_params = {
    'use_extra_x': 1,
    'n_hidden': 1000,
    'n_layers': 3,
    'hidden_p': 0.075,
    'input_p': 0.1,
    'weight_p': 0.125,
    'drops': [0.1] * 4,
    'alpha': 2.,
    'beta': 1.,
    'clip': 0.5,
    'loss_func': DualLoss(n_cont, n_cat, 0.1)
}

hyper_params['layers'] = [3*(context.n_cont) + int(hyper_params['use_extra_x']) * 3, 1000, 13]
learner = get_new_model(context, databunch, hyper_params)

learner.fit(60, lr=1e-1)

In [ ]:
lr = 1e-2
time_str = time.strftime("%Y-%m-%d_%H:%M:%S")
save_name = '_'.join(['final-no-extra-day', str(hyper_params['n_hidden']),context.exp_name,time_str])
callbacks = [SaveModelCallback(learner, name=save_name , every='improvement', monitor='MAPD', mode='min')]
learner.fit(50, lr, callbacks=callbacks)
learner.recorder.plot_metrics()

In [ ]:
xb_valid, yb_valid = ni(databunch.train_dl)
yb_valid = to_np(yb_valid)

In [ ]:
model = learner.model
model.eval()
model.reset()
pb_valid = model(*xb_valid)[0]

# pb = denormalize(pb[0], *context.stat_y)
# pb = pb[0]
# pb = to_np(pb)
# pb_valid = pb_valid[0]
# pb_valid = to_np(pb_valid)

In [ ]:
denormalize(yb_valid, *context.stat_y)[:,:2].argmin()
denormalize(yb_valid, *context.stat_y)[:,:2][4154//2-5]

In [ ]:
hyper_params['layers']

In [ ]:
p_cont, p_cat = pb_valid[:,:n_cont], pb_valid[:,n_cont:]

cat_idx_zip = [0] + hyper_params['loss_func'].cat_idx.tolist()
p_cat_tasks = []
for i,j in zip(cat_idx_zip[:-1], cat_idx_zip[1:]):
    p_cat_tasks.append(to_np(p_cat[:,i:j]))

p1, p2 = [o.argmax(1) for o in p_cat_tasks]

In [ ]:
class_names = cat_columns[0].cat.categories

y_test = yb_valid[:,-2].astype(np.int)
y_pred = p1
plot_confusion_matrix(y_test, y_pred, classes=class_names,
                      title='Confusion matrix, without normalization')
plot_confusion_matrix(y_test, y_pred, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

In [ ]:
class_names = cat_columns[1].cat.categories
y_test = yb_valid[:,-1].astype(np.int)
y_pred = p2
plot_confusion_matrix(y_test, y_pred, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')
set_ax_font(plt.gca())